<a href="https://colab.research.google.com/github/100jy/voice_competition/blob/master/%EB%8D%B0%EC%9D%B4%EC%BD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

구글 드라이브 연동

In [ ]:
#구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# 드라이브내 파일 목록 확인
!ls '/content/drive/My Drive'

'음성 004_sd.m4a'
'음성 005_sd.m4a'
'국민은행 (1).docx'
 삼성1.gdoc
 복싱시합1.mp4
 삼성1.png
 삼성2.gdoc
 삼성2.jpg
 복싱시합2.mp4
 삼성3.gdoc
 삼성3.jpg
 복싱시합3.mp4
'9강 manipulation.hwp'
'Anyfile Notepad Files'
'Colab Notebooks'
 data
 포스코.docx
'제목없는 문서.gdoc'
'학습을 위한 영상매체 이용에 대한 만족도와 큐레이션 서비스에 대한 수요조사.gform'
'부산대학생들의 소비성향 분석을 위한 설문조사.gform'
'할 일 목록.gsheet'
'[멘티]학부 연구생 지원서.hwp'
'KakaoTalkDownload (1).zip'
 KakaoTalkDownload.zip
 시합.mp4
 자기소개서_백주엽_샘플.pdf
'대략적인 구조.png'
 준비.rtf
'sample_submission.csv의 사본'
'tensorflow manipulation.ipynb'


모듈 임포트

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import tensorflow as tf
import keras
from tqdm import tqdm
from glob import glob
from scipy.io import wavfile
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
import librosa

# 음성에서 정형 데이터화

1. 음성신호 상태로 이용하는 방법 
2. MFCC이용하는 방법
3. spectogram 이용하는 방법

In [ ]:
# 데이터 로드 함수 정의 
def make_data(datalist, feature = 'signal'):

  arr = []
  #signal data
  if feature == 'signal':
    for file in tqdm(datalist):
      fs, data = wavfile.read(file)
      arr.append(data)
  
  #spectogam data
  elif feature == 'spectogram':
    for file in tqdm(datalist):
      x, sr = librosa.load(file)
      arr.append(x)
  
  #MFCC data
  elif feature == 'MFCC':
    for file in tqdm(datalist):
      x, sr = librosa.load(file)
      arr.append(x)
  
  else:
    raise ValueError

  result = np.array(arr)
    
  return result 

VAD하면 성능이 더 좋지 않을까???


In [ ]:
#signal 형태로 사용
raw_data = glob('data/train/*.wav')
#signal
signal = make_data(raw_data)
pd.DataFrame(signal).to_pickle('data/x_trian.pickle')
#specto
specto = make_data(raw_data, feature =  'spectogram')
pd.DataFrame(specto).to_pickle('data/x_trian_spectogram.pickle')
#MFCC
MFCC = make_data(raw_data, feature =  'MFCC')
pd.DataFrame(MFCC).to_pickle('data/x_trian_MFCC.pickle')
#feature = 
#x_train = pd.read_csv('data/train/x_train_{}.pickle'.format(feature))

In [ ]:
#signal 형태로 사용
x_train = glob('data/train/*.wav')
x_train = make_data(x_train)

# pickle로 저장
pd.DataFrame(x_train).to_pickle('data/x_trian.pickle')

# (0,1)로 정규화
## 작은 신호 무시되서 좋은 방법 아닐들...
### 로그변환 같은 방법이 더 좋을듯...
x_train = x_train/30000 

# 모델 넣기위한 reshape
## 채널 늘려보는 것도 의미있을듯...
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)

y_train = pd.read_csv('data/train_answer.csv',index_col=0)
y_train = y_tarin.values

x_train.shape, y_train.shape

In [ ]:
#CNN
model = Sequential()
model.add(Conv1D(16,32,activation='relu', input_shape = (x_train.shape[1], x_train.shape[2])))
model.add(MaxPooling1D())
model.add(Conv1D(16,32,activation='relu')
model.add(MaxPooling1D()))
model.add(Conv1D(16,32,activation='relu'))
model.add(MaxPooling1D())
model.add(Conv1D(16,32,activation='relu'))
model.add(MaxPooling1D())
model.add(Conv1D(16,32,activation='relu'))
model.add(Flatten())

#fully connected layer
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(30, activation='softmax'))
model.compile(loss=keras.losses.KLDivergence(), optimizer = 'adam')

In [ ]:
model_path = 'model/'
if not os.path.exists(model_path):
  os.mkdir(model_path)

# validattion 기준 모델 갱신
model_file = model_path + 'epoch_{epoch:03d}_val_{val_loss:3f}.hdf5'
checkpoint = ModelCheckpoint(filepath = model_file, monitor = 'val_loss', verbose = 1, save_best_only =True)

#10회간 validatation 좋아지지 않으면 early stop
early_stop = EarlyStopping(monitor = 'val_loss', patience = 10)

history = model.fit(x_train, y_train, epoch = 100, batchsize = 256, validation_split = 0.2, shuffle = True,
                    callbacks = [cheakpoint, early_stopping])

In [ ]:
# 훈련 결과를 확인합니다.
plt.plot(history.epoch, history.history['loss'], '-o', label='training_loss')
plt.plot(history.epoch, history.history['val_loss'], '-o', label='validation_loss')
plt.legend()
plt.xlim(left=0)
plt.xlabel('epochs')
plt.ylabel('loss')
plt.show()

전처리 필요성, 모델 CNN 이용
